In [ ]:
import os
import sys

import h5py
import numpy as np
import seaborn as sns

sys.path.insert(0, os.path.abspath("../.."))

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.collections import LineCollection

from transformers.data_utils.bb_data_loading import load_list
from transformers.training.jax_utils import cross_ent_loss
from transformers.training.utils import load_pickle

In [ ]:
t0012_list = load_list("../t0012/preference_data_1/day_list.txt")

In [ ]:
with h5py.File("../t0012/preference_data_1/t0012.hdf5") as f:
    sts_2 = f["states_2"][:]
    acts_2 = f["actions_2"][:]
    ts_2 = f["timesteps_2"][:]
    am_2 = f["attn_mask_2"][:]

    sts = f["states"][:]
    acts = f["actions"][:]
    ts = f["timesteps"][:]
    am = f["attn_mask"][:]

In [ ]:
pr = []
it = 10
interval_list = list(range(0, len(t0012_list), it))
interval_list = interval_list + [interval_list[-1] + it]
interval_list = list(zip(interval_list, interval_list[1:]))


def find_intervals(n, interval_list):
    intervals = []
    for i, j in interval_list:
        intervals.append((i, j))
        if n >= i and n < j:
            return intervals


n_model = load_pickle("../t0012/results/pt_rewards/best_model.pkl")["model"]

nbr = []
for n, m in interval_list:
    inb = np.all((sts_2[:, :, 15] >= n) & (sts_2[:, :, 15] < m), axis=1)
    s_2 = sts_2[inb, ...]
    if s_2.shape[0] > 0:
        a_2 = acts_2[inb, ...]
        t_2 = ts_2[inb, ...]
        atm_2 = am_2[inb, ...]

        s_1 = sts[inb, ...]
        a_1 = acts[inb, ...]
        t_1 = ts[inb, ...]
        atm_1 = am[inb, ...]

        s_2[:, :, 15] = np.where(atm_2 != 0, m, s_2[:, :, 15])
        s_1[:, :, 15] = np.where(atm_1 != 0, m, s_1[:, :, 15])
        n_pred_2, n_weights_2 = n_model._train_state.apply_fn(
            n_model._train_state.params,
            s_2,
            a_2,
            t_2,
            training=False,
            attn_mask=atm_2,
        )
        n_pred, n_weights = n_model._train_state.apply_fn(
            n_model._train_state.params,
            s_1,
            a_1,
            t_1,
            training=False,
            attn_mask=atm_1,
        )
        B, T, _ = s_1.shape
        trans_pred_1 = n_pred["weighted_sum"]
        trans_pred_2 = n_pred_2["weighted_sum"]

        sum_pred_1 = np.nanmean(trans_pred_1.reshape(B, T), axis=1).reshape(-1, 1)
        sum_pred_2 = np.nanmean(trans_pred_2.reshape(B, T), axis=1).reshape(-1, 1)

        m_p = 1 / (np.exp(sum_pred_1 - sum_pred_2) + 1)

        s_2[:, :, 15] = np.where(atm_2 != 0, n, s_2[:, :, 15])
        s_1[:, :, 15] = np.where(atm_1 != 0, n, s_1[:, :, 15])
        n_pred_2, n_weights_2 = n_model._train_state.apply_fn(
            n_model._train_state.params,
            s_2,
            a_2,
            t_2,
            training=False,
            attn_mask=atm_2,
        )
        n_pred, n_weights = n_model._train_state.apply_fn(
            n_model._train_state.params,
            s_1,
            a_1,
            t_1,
            training=False,
            attn_mask=atm_1,
        )
        B, T, _ = s_1.shape
        trans_pred_1 = n_pred["weighted_sum"]
        trans_pred_2 = n_pred_2["weighted_sum"]

        sum_pred_1 = np.nanmean(trans_pred_1.reshape(B, T), axis=1).reshape(-1, 1)
        sum_pred_2 = np.nanmean(trans_pred_2.reshape(B, T), axis=1).reshape(-1, 1)

        n_p = 1 / (np.exp(sum_pred_1 - sum_pred_2) + 1)
        nbr.append(np.mean(m_p - n_p))

pr = []
for i in range(len(t0012_list)):
    intervals = find_intervals(i, interval_list)
    e_sum = 0.0
    for j in range(len(intervals)):
        if j < len(nbr):
            e_sum += nbr[j]
    pr.append(e_sum)

In [ ]:
pr = np.array(pr)
pr = pr - np.mean(pr)

In [ ]:
fig, ax = plt.subplots()
ax.plot(pr)
ax.set_title("ALE Plot for Day in Experiment")
ax.set_xlabel("Day")
ax.set_ylabel("ALE")
plt.show()